In [1]:
#importer les packages 
import numpy as np
import pickle
from sklearn.metrics import precision_score, recall_score, f1_score
import time

In [4]:
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('TkAgg')  # Replace 'TkAgg' with the appropriate backend name if needed
import seaborn as sns


# Charger le dataset Titanic
df = pd.read_csv('train.txt')

# Encoder les variables catégorielles
label_encoder = LabelEncoder()
categorical_vars = ['Embarked', 'Cabin']
for var in categorical_vars:
    df[var] = label_encoder.fit_transform(df[var].astype(str))

# Créer une copie du DataFrame pour la manipulation
df_imputed = df.copy()

# Variables à imputer
vars_with_missing_values = ['Age', 'Embarked', 'Cabin']

# Imputer les valeurs manquantes avec KNNImputer
imputer = KNNImputer(n_neighbors=5)
df_imputed[vars_with_missing_values] = imputer.fit_transform(df_imputed[vars_with_missing_values])

# Variables incomplètes après imputation
incomplete_vars = ['Embarked', 'Cabin']

# Créer un modèle prédictif pour chaque variable incomplète
for var in incomplete_vars:
    complete_vars = [x for x in incomplete_vars if x != var]
    complete_data = df_imputed.loc[df_imputed[var].notnull(), complete_vars]
    incomplete_data = df_imputed.loc[df_imputed[var].isnull(), complete_vars]

    if incomplete_data.shape[0] > 0:
        target = df_imputed.loc[df_imputed[var].notnull(), var]
        model = HistGradientBoostingRegressor()
        model.fit(complete_data, target)
        predicted_values = model.predict(incomplete_data)
        df_imputed.loc[df_imputed[var].isnull(), var] = predicted_values

# Vérification des modifications
missing_values_after = df_imputed.isnull().sum()
print(missing_values_after)

# Configuration des options d'affichage pour afficher toutes les lignes et toutes les colonnes
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)  # Définir la largeur d'affichage sur None pour éviter la troncature des colonnes

# Afficher le dataset après élimination des valeurs manquantes
# print(df_imputed)
df_imputed.to_csv('train_imputed.csv', index=False)




'''Partie de elimination des valeures aberantes '''
# Chargement des données Titanic
data = pd.read_csv('train_imputed.csv')



# Détection et traitement des valeurs aberrantes pour la caractéristique "Age"
feature = 'Age'

# Visualisation améliorée avec un boxplot et un histogramme
fig, axes = plt.subplots(1, 2, figsize=(12, 6))
sns.boxplot(data[feature], ax=axes[0])
axes[0].set_ylabel(feature)
sns.histplot(data[feature], kde=True, ax=axes[1])
axes[1].set_xlabel(feature)
plt.show()

# Détection des valeurs aberrantes
Q1 = data[feature].quantile(0.25)
Q3 = data[feature].quantile(0.75)
IQR = Q3 - Q1
upper_bound = Q3 + 1.5 * IQR
lower_bound = Q1 - 1.5 * IQR

outliers = data[(data[feature] > upper_bound) | (data[feature] < lower_bound)]
print(f"Valeurs aberrantes pour la caractéristique '{feature}':")
print(outliers)

# Traitement des valeurs aberrantes (remplacement par la médiane)
median_age = data[feature].median()
data.loc[outliers.index, feature] = median_age

# Affichage des lignes avec les valeurs aberrantes remplacées
print("Lignes avec les valeurs aberrantes remplacées:")
print(data.loc[outliers.index])

# Affichage du jeu de données après traitement des valeurs aberrantes
print("Jeu de données après traitement des valeurs aberrantes:")
data.to_csv('resultat.csv', index=False)

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
Duplicate      0
dtype: int64
Valeurs aberrantes pour la caractéristique 'Age':
Empty DataFrame
Columns: [PassengerId, Survived, Pclass, Name, Sex, Age, SibSp, Parch, Ticket, Fare, Cabin, Embarked, Duplicate]
Index: []
Lignes avec les valeurs aberrantes remplacées:
Empty DataFrame
Columns: [PassengerId, Survived, Pclass, Name, Sex, Age, SibSp, Parch, Ticket, Fare, Cabin, Embarked, Duplicate]
Index: []
Jeu de données après traitement des valeurs aberrantes:


In [5]:
pwd

'C:\\Users\\Meriem\\Desktop\\Netoyage des Donnees'

In [9]:
mydata = pd.read_csv("resultat.csv")

In [10]:
mydata


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Duplicate
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,5.0,2.0,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence BriggsTha...",female,38.0,1,0,PC 17599,71.2833,2.0,0.0,0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,5.0,2.0,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,1.0,2.0,1
4,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,1.0,2.0,1
5,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,5.0,2.0,0
6,6,0,3,"Moran, Mr. James",male,29.2,0,0,330877,8.4583,5.0,1.0,0
7,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,3.0,2.0,1
8,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,3.0,2.0,1
9,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,5.0,2.0,0


In [11]:
#voir les valeurs manquantes 
mydata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  25 non-null     int64  
 1   Survived     25 non-null     int64  
 2   Pclass       25 non-null     int64  
 3   Name         25 non-null     object 
 4   Sex          25 non-null     object 
 5   Age          25 non-null     float64
 6   SibSp        25 non-null     int64  
 7   Parch        25 non-null     int64  
 8   Ticket       25 non-null     object 
 9   Fare         25 non-null     float64
 10  Cabin        25 non-null     float64
 11  Embarked     25 non-null     float64
 12  Duplicate    25 non-null     int64  
dtypes: float64(4), int64(6), object(3)
memory usage: 2.7+ KB


In [12]:
mydata.isnull().sum().sort_values(ascending=False)

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
Duplicate      0
dtype: int64

In [13]:
mydata.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Cabin,Embarked,Duplicate
count,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000
mean,11.280000,0.520000,2.360000,30.968000,0.680000,0.600000,24.573496,4.160000,1.680000,0.320000
std,6.605049,0.509902,0.860233,16.457286,0.988264,1.414214,18.292385,1.545962,0.690411,0.476095
min,1.000000,0.000000,1.000000,2.000000,0.000000,0.000000,7.225000,0.000000,0.000000,0.000000
25%,6.000000,0.000000,2.000000,21.000000,0.000000,0.000000,8.458300,4.000000,2.000000,0.000000
50%,11.000000,1.000000,3.000000,31.000000,0.000000,0.000000,18.000000,5.000000,2.000000,0.000000
75%,16.000000,1.000000,3.000000,38.000000,1.000000,0.000000,31.275000,5.000000,2.000000,1.000000
max,26.000000,1.000000,3.000000,58.000000,4.000000,5.000000,71.283300,5.000000,2.000000,1.000000


In [14]:
mydata.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Cabin,Embarked,Duplicate
count,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000
mean,11.280000,0.520000,2.360000,30.968000,0.680000,0.600000,24.573496,4.160000,1.680000,0.320000
std,6.605049,0.509902,0.860233,16.457286,0.988264,1.414214,18.292385,1.545962,0.690411,0.476095
min,1.000000,0.000000,1.000000,2.000000,0.000000,0.000000,7.225000,0.000000,0.000000,0.000000
25%,6.000000,0.000000,2.000000,21.000000,0.000000,0.000000,8.458300,4.000000,2.000000,0.000000
50%,11.000000,1.000000,3.000000,31.000000,0.000000,0.000000,18.000000,5.000000,2.000000,0.000000
75%,16.000000,1.000000,3.000000,38.000000,1.000000,0.000000,31.275000,5.000000,2.000000,1.000000
max,26.000000,1.000000,3.000000,58.000000,4.000000,5.000000,71.283300,5.000000,2.000000,1.000000


In [15]:
#elimination du colonne 
mydata = mydata.drop('Cabin', axis=1)

In [16]:
mydata.to_csv('test.csv', index=False)


In [17]:
mydata


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Duplicate
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,2.0,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence BriggsTha...",female,38.0,1,0,PC 17599,71.2833,0.0,0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,2.0,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,2.0,1
4,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,2.0,1
5,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,2.0,0
6,6,0,3,"Moran, Mr. James",male,29.2,0,0,330877,8.4583,1.0,0
7,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,2.0,1
8,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,2.0,1
9,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,2.0,0


In [18]:
age_median = mydata['Age'].median()
age_median

31.0

In [19]:
mydata['Age'].fillna(age_median, inplace=True)
#mydata.to_csv('train_p.csv', index=False)
mydata.to_csv('test.csv', index=False)


In [20]:
mydata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  25 non-null     int64  
 1   Survived     25 non-null     int64  
 2   Pclass       25 non-null     int64  
 3   Name         25 non-null     object 
 4   Sex          25 non-null     object 
 5   Age          25 non-null     float64
 6   SibSp        25 non-null     int64  
 7   Parch        25 non-null     int64  
 8   Ticket       25 non-null     object 
 9   Fare         25 non-null     float64
 10  Embarked     25 non-null     float64
 11  Duplicate    25 non-null     int64  
dtypes: float64(3), int64(6), object(3)
memory usage: 2.5+ KB


In [21]:
# Find the most frequent value in the "Embarked" column
most_frequent_embarked = mydata['Embarked'].mode()[0]

In [22]:
most_frequent_embarked

2.0

In [23]:
# Fill the missing values with the most frequent value
mydata['Embarked'].fillna(most_frequent_embarked, inplace=True)
# Save the modified dataset
mydata.to_csv('test.csv', index=False)

In [24]:
mydata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  25 non-null     int64  
 1   Survived     25 non-null     int64  
 2   Pclass       25 non-null     int64  
 3   Name         25 non-null     object 
 4   Sex          25 non-null     object 
 5   Age          25 non-null     float64
 6   SibSp        25 non-null     int64  
 7   Parch        25 non-null     int64  
 8   Ticket       25 non-null     object 
 9   Fare         25 non-null     float64
 10  Embarked     25 non-null     float64
 11  Duplicate    25 non-null     int64  
dtypes: float64(3), int64(6), object(3)
memory usage: 2.5+ KB


In [25]:
import recordlinkage

In [26]:
start_time = time.time()
#Prepare the dataset for deduplication:
# Convert relevant columns to string type for comparison
mydata['Name'] = mydata['Name'].astype(str)
mydata['Ticket'] = mydata['Ticket'].astype(str)
# Add an index to the dataset
# mydata = mydata.drop(columns=['level_0'])
mydata = mydata.reset_index()

In [27]:
# Define the indexing step using sorted neighborhood method
indexer = recordlinkage.Index()
indexer.sortedneighbourhood(left_on='Name', window=3)

<Index>

In [28]:
# Perform the indexing
pairs = indexer.index(mydata)

In [29]:
# Define the comparison step using string similarity on Name and Ticket columns
compare = recordlinkage.Compare()
compare.string('Name', 'Name', threshold=0.85, label='Name')
compare.string('Ticket', 'Ticket', threshold=0.85, label='Ticket')

<Compare>

In [30]:
# Perform the comparison
features = compare.compute(pairs, mydata)
features

,,Name,Ticket
3,2,0.0,0.0
4,2,0.0,0.0
7,6,0.0,0.0
8,6,0.0,0.0
11,9,0.0,0.0
13,0,0.0,0.0
17,10,0.0,0.0
18,10,0.0,0.0
19,12,0.0,0.0
21,16,0.0,0.0


In [31]:
threshold = 1.0

In [32]:
# Identify potential duplicates based on the similarity scores
potential_duplicates = features[features.sum(axis=1) > threshold]
duplicate_pairs = potential_duplicates.index.get_level_values(1)
duplicate_records = mydata.loc[duplicate_pairs]
duplicate_records
end_time = time.time()
duplicate_records


,index,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Duplicate
3,3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,2.0,1
7,7,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,2.0,1
17,17,16,1,2,"Hewlett, Mrs. (Mary D Kingcome)",female,55.0,0,0,248706,16.0000,2.0,1
21,21,19,0,3,"Vander Planke, Mrs. Julius (Emelia Maria Vande...",female,31.0,1,0,345763,18.0000,2.0,1


In [33]:
# mydata.loc[mydata['Name'] == "Thayer, Mr. John Borland Jr"]
#mydata.loc[mydata['Name'].str.startswith("Thayer")]


In [34]:
end_time-start_time

4.573323488235474